# Supervised Fine‑Tuning with LoRA (Parameter‑Efficient)
We’ll adapt a small causal LM with LoRA on a toy instruction dataset.

In [ ]:
!pip -q install -U transformers datasets accelerate peft bitsandbytes trl


In [ ]:
import torch, os, json, random
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


In [ ]:
# Build a tiny instruction dataset inline (for demo). Replace with your own JSONL later.
examples = [
    {"instruction": "Summarize the following text", "input": "Large language models can be adapted efficiently with LoRA.", "output": "LoRA lets you adapt LLMs efficiently."},
    {"instruction": "Translate to French", "input": "Parameter-efficient fine-tuning", "output": "Ajustement fin efficace en paramètres"},
    {"instruction": "Write a short title", "input": "A tutorial on PEFT methods", "output": "PEFT Tutorial"},
    {"instruction": "Extract entities", "input": "OpenAI released a new model in 2025", "output": "ORG: OpenAI; DATE: 2025"},
] * 64  # duplicate to have a few hundred rows
random.shuffle(examples)
with open("toy_instruct.jsonl", "w") as f:
    for ex in examples:
        f.write(json.dumps(ex) + "\n")


In [ ]:
from datasets import load_dataset
ds = load_dataset("json", data_files="toy_instruct.jsonl", split="train")
def format_row(ex):
    if ex.get("input"):
        return f"### Instruction\n{ex['instruction']}\n\n### Input\n{ex['input']}\n\n### Response\n{ex['output']}"
    else:
        return f"### Instruction\n{ex['instruction']}\n\n### Response\n{ex['output']}"


In [ ]:
model_name = "distilgpt2"  # small for demo
tok = AutoTokenizer.from_pretrained(model_name)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# 4-bit load to save memory (optional)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

peft_config = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["c_attn", "c_proj"],  # works for GPT2-family
    bias="none",
    task_type="CAUSAL_LM",
)


In [ ]:
cfg = SFTConfig(
    output_dir="sft-lora-demo",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=20,
    save_steps=200,
    max_seq_length=256,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    bf16=True if torch.cuda.is_available() else False,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    args=cfg,
    train_dataset=ds,
    formatting_func=lambda batch: [format_row(x) for x in batch],
    tokenizer=tok,
    peft_config=peft_config
)
trainer.train()
trainer.model.save_pretrained("sft-lora-demo/model")
tok.save_pretrained("sft-lora-demo/tokenizer")


In [ ]:
# Test generation
from transformers import pipeline
pipe = pipeline("text-generation", model="sft-lora-demo/model", tokenizer="sft-lora-demo/tokenizer", device=0 if torch.cuda.is_available() else -1)
prompt = "### Instruction\nTranslate to French\n\n### Input\nFine-tuning with LoRA is efficient.\n\n### Response\n"
out = pipe(prompt, max_new_tokens=60, do_sample=True, temperature=0.7)[0]["generated_text"]
print(out)
